In [1]:
import os
import zipfile
import pickle
from mgz import header, body
from mgz.model import parse_match
import os


In [2]:
def print_txt(msg: str, new_line = False):
    print(msg, end = '\r')
    text_file = open("Outputs/Output_Unzipper.txt", "w")
    if new_line:
        text_file.write("\n")
        
    text_file.write(msg)
    text_file.close()

In [3]:
#settings
zip_path = "data/scraped_matches/zipped_recordfiles/"
unzip_path = "data/scraped_matches/unzipped_recordfiles/"

testparse = True
delete = False 

In [4]:


#load already parseable recordfiles
with open('data/scraped_matches/unzipper/parseable_fns.pkl', 'rb') as f:
    old_parseable_fns = pickle.load(f)
    

In [ ]:

#unzip and testparse files
directory = os.fsencode(zip_path)
file_list = os.listdir(directory)

error_files = []
unzipped_matches = []
error_ids = []
parseable_fns = old_parseable_fns
unparseable_fns = []

unzipable = 0
unparseable = 0
other = 0

if testparse:
    for index, file in enumerate(file_list):
        filename = os.fsdecode(file)
        if filename not in old_parseable_fns:
            match_id = filename.partition("=")[2].partition("&")[0]

            #unzip
            try:
                with zipfile.ZipFile(zip_path + "{}".format(filename),"r") as zip_ref:
                    zip_ref.extractall(unzip_path)

                #testparse and delete
                try:
                    record_fn = 'AgeIIDE_Replay_{}.aoe2record'.format(match_id)

                    with open(unzip_path + record_fn, 'rb') as data:
                        eof = os.fstat(data.fileno()).st_size
                        body.meta.parse_stream(data)
                        while data.tell() < eof:
                            body.operation.parse_stream(data)

                    with open(unzip_path + record_fn, 'rb') as data:
                                match = parse_match(data)

                    os.remove(unzip_path + record_fn)
                    parseable_fns.append(filename)

                except RuntimeError as e:
                    try:
                        error_ids.append(match_id)
                        os.remove(unzip_path + record_fn)

                        if e.args[0] == "could not parse":
                            unparseable +=1
                            unparseable_fns.append(filename)
                        else:
                            other +=1
                    except: 
                        other +=1

            except:
                unzipable +=1
                error_files.append(filename)
                error_ids.append(match_id)


            if index % 100 == 0:
                print_txt("Parsed: {} - Unzipable: {} - Unparseable: {} - other: {}".format(index,unzipable,unparseable,other))
                output = open('data/scraped_matches/unzipper/unzipper_error_ids.pkl' , 'wb')
                pickle.dump(error_ids, output)
                output.close()
                output = open('data/scraped_matches/unzipper/error_files.pkl' , 'wb')
                pickle.dump(error_files, output)
                output.close()
                output = open('data/scraped_matches/unzipper/parseable_fns_ot.pkl' , 'wb')
                pickle.dump(parseable_fns, output)
                output.close()
                output = open('data/scraped_matches/unzipper/unparseable_fns.pkl' , 'wb')
                pickle.dump(unparseable_fns, output)
                output.close()
    
    #final persistation
    output = open('data/scraped_matches/unzipper/parseable_fns.pkl' , 'wb')
    pickle.dump(parseable_fns, output)
    output.close()
    output = open('data/scraped_matches/unzipper/unparseable_fns.pkl' , 'wb')
    pickle.dump(unparseable_fns, output)
    output.close()

In [ ]:
print(len(parseable_fns),len(unparseable_fns),len(error_files))

In [ ]:
"""
#unzip all files
directory = os.fsencode(zip_path)
file_list = os.listdir(directory)
files = len(file_list)

searched_files = 0
error_files = []
unzipped_matches = []


#find already unziped matches
for file in os.listdir(unzip_path):
    filename = os.fsdecode(file)
    match_id = filename[15:-11]
    unzipped_matches.append(match_id)


for index, file in enumerate(file_list):
    filename = os.fsdecode(file)
    match_id = filename.partition("=")[2].partition("&")[0]
   
    if match_id not in unzipped_matches:
        try:
            with zipfile.ZipFile(zip_path + "{}".format(filename),"r") as zip_ref:
                zip_ref.extractall(unzip_path)
        except:
            error_files.append(filename)

    searched_files +=1
    
    print(str(searched_files) + "/" +str(files)+ " were unziped - error in " + str(len(error_files)) + " files", end = '\r')
    if index % 1000 == 0:
        print_txt(str(searched_files) + "/" +str(files)+ " were unziped - error in " + str(len(error_files)) + " files")
      
"""

In [ ]:
"""
output = open('data/scraped_matches/unzipper_error_files.pkl' , 'wb')
pickle.dump(error_files, output)
output.close()
"""

In [ ]:
"""

#check if files can be parsed
from mgz import header, body
from mgz.model import parse_match
import os


unparseable_files = []
unparseable = 0
other = 0

files = os.listdir(unzip_path)

for index,file in enumerate(files):
    try:
        with open(unzip_path + file, 'rb') as data:
            eof = os.fstat(data.fileno()).st_size
            body.meta.parse_stream(data)
            while data.tell() < eof:
                body.operation.parse_stream(data)

        with open(unzip_path + file, 'rb') as data:
                    match = parse_match(data)
    
    except RuntimeError as e:
        try:
            unparseable_files.append(file)
            if e.args[0] == "could not parse":
                unparseable +=1
            else:
                other +=1
        except: 
            other +=1
    
    except:
        other +=1
    if index % 100 == 0:
        print_txt(str(index)+'/'+str(len(files)) + " Parsed - Uparseable: " + str(unparseable) + " - other Errors: " +str(other))
        output = open('data/scraped_matches/unzipper_unparsable_files.pkl' , 'wb')
        pickle.dump(unparseable_files, output)
        output.close()
        
output = open('data/scraped_matches/unzipper_unparsable_files.pkl' , 'wb')
pickle.dump(unparseable_files, output)
output.close()

"""

In [ ]:
if delete:
    #unzip and testparse files
    directory = os.fsencode(zip_path)
    file_list = os.listdir(directory)

    error_files = []

    #load errorfiles
    with open('data/scraped_matches/unzipper/parseable_fns.pkl', 'rb') as f:
            parseable_fns = pickle.load(f)


    for index, file in enumerate(file_list):
        filename = os.fsdecode(file)
        if filename not in parseable_fns:
            error_files.append(filename)

        if index % 1000 == 0:
            print(str(index)+str(len(error_files)) + " - " + len(error_files), end = '\r')


    print(len(error_files))
    print(len(parseable_fns))


    if False:
        pass
